In [156]:
import numpy as np
import pandas as pd

In [157]:
df_bak = pd.read_excel('../01-Data/ordens_pre_process.xlsx')

In [158]:
df = df_bak.drop(['Unnamed: 0'], axis=1)
df.columns = df.columns.str.replace(' ', '_')

In [159]:
# df = df[df['Inner'] == 1]
df = df[df['Loja'] == 6]
df = df[df['Classe_de_onda'] == 4]
df = df[df['Tipo_de_buffer'] == 'CX']

In [160]:
print(df['Inner'].unique())

[1 2]


In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53 entries, 3935 to 31314
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Ordem_de_produção          53 non-null     int64         
 1   Data_da_ordem_de_produção  53 non-null     datetime64[ns]
 2   Produto                    53 non-null     int64         
 3   Comprimento_unit           53 non-null     int64         
 4   Largura_unit               53 non-null     int64         
 5   Altura_unit                53 non-null     int64         
 6   Peso_unit                  53 non-null     float64       
 7   Peças                      53 non-null     int64         
 8   Caixa_padrão               53 non-null     object        
 9   Loja                       53 non-null     int64         
 10  Classe_de_onda             53 non-null     int64         
 11  Tipo_de_buffer             53 non-null     object        
 12  Item_pai 

In [165]:
import pandas as pd

def first_fit_decreasing(df, bin_capacity_vol, bin_capacity_weight, bin_item_limit):
    # Sort items in decreasing order of volume
    df_sorted = df.sort_values(by='Volume_total', ascending=False)
    
    # Initialize bins list
    bins = []
    
    # Flag to track if a split was made
    split_made = True
    items_to_drop = []
    df_sorted = df_sorted.reset_index(drop=True)
    items_to_add = []

      
    while len(df_sorted) != 0:
        split_made = False

        df_sorted = df_sorted[~df_sorted.index.isin(items_to_drop)]

        # Add needed items to df_sorted
        if len(items_to_add) != 0:
            df_sorted = pd.concat([df_sorted, pd.DataFrame(items_to_add)])
        items_to_add = []
        
        df_sorted = df_sorted.reset_index(drop=True)
        items_to_drop = []
        
        # Make a copy of the sorted DataFrame
        df_sorted_copy = df_sorted.copy()
        # Iterate over items in the copy of the sorted DataFrame
        for index, item in df_sorted_copy.iterrows():
            # Flag to check if the item is placed
            placed = False
            
            # Try to place the item in existing bins
            for bin in bins:
                if (bin['Volume'] + item['Volume_total']) <= bin_capacity_vol \
                   and (bin['Peso_total'] + item['Peso_total']) <= bin_capacity_weight \
                   and bin['Amt.Items'] + item['Peças'] <= bin_item_limit:
                    bin['Items'].append(index)
                    bin['Volume'] += item['Volume_total']
                    bin['Peso_total'] += item['Peso_total']
                    bin['Amt.Items'] += item['Peças']

                    # Remove the item from the original DataFrame
                    items_to_drop.append(index)

                    placed = True
                    break
            
            # If the item couldn't be placed entirely, reduce it by "inner" means until it fits into a bin
            while not placed and item['Peças'] > item['Inner']:
                # Reduce the item to its inner size
                detached_portion = item.copy()
                detached_portion['Peças'] = item['Inner']
                detached_portion['Volume_total'] = item['Volume_unit'] * item['Inner']
                detached_portion['Peso_total'] = item['Peso_unit'] * item['Inner']
                
                # Update the remaining portion of the item
                item['Peças'] -= item['Inner']
                item['Volume_total'] -= item['Volume_unit'] * item['Inner']
                item['Peso_total'] -= item['Peso_unit'] * item['Inner']
                
                # Concatenate the reduced portion back to the original DataFrame
                items_to_add.append(detached_portion)
                split_made = True
                
                # Try to place the reduced item in existing bins
                for bin in bins:
                    if (bin['Volume'] + item['Volume_total']) <= bin_capacity_vol \
                       and (bin['Peso_total'] + item['Peso_total']) <= bin_capacity_weight \
                       and bin['Amt.Items'] + item['Peças'] <= bin_item_limit:
                        bin['Items'].append(index)  # Append index of the appended reduced item
                        bin['Volume'] += item['Volume_total']
                        bin['Peso_total'] += item['Peso_total']
                        bin['Amt.Items'] += item['Peças']

                        items_to_drop.append(index) # Remove the reduced item from the original DataFrame
                        placed = True
                        break
            
            # If the item still couldn't be placed, create a new bin
            if not placed:
                bins.append({'Items': [index], 'Volume': item['Volume_total'], 'Peso_total': item['Peso_total'], 'Amt.Items': item['Peças']})
                items_to_drop.append(index)
                placed = True
                break
    
    return bins


In [166]:
def old_first_fit_decreasing(df, bin_capacity_vol, bin_capacity_weight, bin_item_limit):
    # Sort items in decreasing order of volume
    df_sorted = df.sort_values(by='Volume_total', ascending=False)
    
    # Initialize bins list
    bins = []
    
    for index, item in df_sorted.iterrows():
        for bin in bins:
            if (bin['Volume'] + item['Volume_total']) <= bin_capacity_vol and \
                (bin['Peso_total'] + item['Peso_total']) <= bin_capacity_weight and \
                bin['Amt.Items'] <= bin_item_limit:
                    bin['Items'].append(index)
                    bin['Volume'] += item['Volume_total']
                    bin['Peso_total'] += item['Peso_total']
                    bin['Amt.Items'] += item['Peças']
                    break
        else:
            # Create new bin
            bins.append({'Items': [index], 'Volume': item['Volume_total'], 'Peso_total': item['Peso_total'], 'Amt.Items': item['Peças']})
    
    return bins



In [167]:
bin_capacity_vol = 83393894.40
bin_capacity_weight = 23
bin_item_limit = 170
packed_bins_new = first_fit_decreasing(df, bin_capacity_vol, bin_capacity_weight, bin_item_limit)
packed_bins_old = old_first_fit_decreasing(df, bin_capacity_vol, bin_capacity_weight, bin_item_limit)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins_new):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print("\nPacked bins (old method):")
for i, bin in enumerate(packed_bins_old):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

Packed bins (new method):
Bin 1: Amt. Items 18 (Total Volume: 82794797 cubic mm | Total Weight: 5.1000kg)
Bin 2: Amt. Items 27 (Total Volume: 83323287 cubic mm | Total Weight: 11.1070kg)
Bin 3: Amt. Items 23 (Total Volume: 82950539 cubic mm | Total Weight: 9.7860kg)
Bin 4: Amt. Items 30 (Total Volume: 81909153 cubic mm | Total Weight: 8.8310kg)
Bin 5: Amt. Items 6 (Total Volume: 27054084 cubic mm | Total Weight: 1.5800kg)

Packed bins (old method):
Bin 1: Amt. Items 20 (Total Volume: 83375243 cubic mm | Total Weight: 5.6400kg)
Bin 2: Amt. Items 24 (Total Volume: 83013681 cubic mm | Total Weight: 7.9740kg)
Bin 3: Amt. Items 21 (Total Volume: 83186828 cubic mm | Total Weight: 10.7000kg)
Bin 4: Amt. Items 27 (Total Volume: 82529145 cubic mm | Total Weight: 8.5450kg)
Bin 5: Amt. Items 12 (Total Volume: 25926963 cubic mm | Total Weight: 3.5450kg)
